In [1]:
import pandas as pd
from datetime import timedelta
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import glob
from pathlib import Path

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import requests
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
# Import CSV Containing IPO Information - Ticker, Industry, # of Shares Offered, Offer Price, 1st Day % Return (IPO Scoop)
ipo_csv = pd.read_csv('IPO Project - Sheet2.csv')

In [3]:
# Drop unnecessary columns
ipo_df = ipo_csv.drop(columns=['1st Day Close'])

# Assign stock ticker as index
ipo_df.set_index('Symbol',inplace=True)

# Create date columns for parsing that represent a 90 day period starting 10days before IPO Offer date
ipo_df['Offer Date'] = pd.to_datetime(ipo_df['Offer Date'])
ten_days = pd.to_timedelta(10,'d')
three_months = pd.to_timedelta(100,'d')

ipo_df['T-10D'] = ipo_df['Offer Date']-ten_days
ipo_df['T-90D'] = ipo_df['Offer Date']-three_months
ipo_df['T+100D'] = ipo_df['Offer Date']+three_months

In [4]:
ipo_df.head(3)

,Industry,Offer Date,Shares (millions),Offer Price,Return,T-10D,T-90D,T+100D
Symbol,,,,,,,,
ABOS,Health Care,2021-07-01,10.0,16.0,0.26,2021-06-21,2021-03-23,2021-10-09
ABSI,Health Care,2021-07-22,12.5,16.0,0.35,2021-07-12,2021-04-13,2021-10-30
ACT,Financials,2021-09-16,13.3,19.0,0.08,2021-09-06,2021-06-08,2021-12-25


In [5]:
ipo_df['T-10D'] = ipo_df['T-10D'].astype(str)
ipo_df['T-90D'] = ipo_df['T-90D'].astype(str)
ipo_df['T+100D'] = ipo_df['T+100D'].astype(str)

In [5]:
# Use Yahoo Finance data to measure returns for above-mentioned 90 day period
for ticker in ipo_df.index:
    data = yf.download('SPY',ipo_df.loc[ticker,'T-90D'],ipo_df.loc[ticker,'T-10D'], progress=False)
    spy_return = (data.iloc[-1,-2] - data.iloc[0,-2])/data.iloc[0,-2]
    ipo_df.loc[ticker,'SPY 90D Return'] = spy_return

    #Use Yahoo Finance data to measure returns for +100 days
    #data2 = yf.download(ticker,ipo_df.loc[ticker,'Offer Date'],ipo_df.loc[ticker,'T+100D'], progress=False)
    #stock_return = (data2.iloc[-1,-2] - data2.iloc[0,-2])/data2.iloc[0,-2]
    #ipo_df.loc[ticker,'100 day Return'] = stock_return    
    
    # Encode Dffer Date to week of year
    ipo_df.loc[ticker,'Offer Date'] = ipo_df.loc[ticker,'Offer Date'].isocalendar()[1]

In [6]:
# Create binary encoded column indicating whether stock saw 10% returns on day of IPO    
returns_bin = []
for pct in ipo_df.Return:
    if pct >= .1:
        returns_bin.append(1)
    else:
        returns_bin.append(0)
        
ipo_df['10% Returns?'] = returns_bin

#Encoding "Industry" Column
le = LabelEncoder()
le.fit(ipo_df["Industry"])
ipo_df["Industry"] = le.transform(ipo_df["Industry"])

# Drop unecessary date columns
ipo_df.drop(columns=['T-10D','T-90D','T+100D'], inplace=True)

ipo_df.head(3)

,Industry,Offer Date,Shares (millions),Offer Price,Return,SPY 90D Return,10% Returns?
Symbol,,,,,,,
ABOS,4,26,10.0,16.0,0.26,0.068748,1
ABSI,4,29,12.5,16.0,0.35,0.058337,1
ACT,3,37,13.3,19.0,0.08,0.076448,0


In [7]:
# Use glob to create categorized lists of Yahoo Finance CSV's
bs_list = glob.glob("*_annual_balance-sheet.csv")
cf_list = glob.glob("*_annual_cash-flow.csv")
af_list = glob.glob("*_annual_financials.csv")
list_of_lists = [bs_list,cf_list,af_list]

In [8]:
# Clean and organize CSV
def step_1(frame):
    frame_df = frame.set_index('name')
    
    # Delete TTM column as this data may be more recent than date of IPO
    if 'ttm' in frame_df.columns: 
        frame_df.drop(columns = 'ttm',inplace=True)
    else:
        pass
    
    # Keep only latest annual financial data from year prior to IPO
    frame_df = frame_df.iloc[:,0]
    
    # Fill NaN's
    frame_df = pd.DataFrame(frame_df.fillna(0))
    
    # Transpose DataFrame
    frame_df = frame_df.T
    
    # Standardize index
    frame_df = frame_df.reset_index()
    
    #Remove '\t...' from column names
    frame_df.columns = frame_df.columns.str.strip()
    
    return frame_df

In [9]:
#Create dataframe for CSV per respective category
def step_2(list):
    df = pd.DataFrame()
    
    # Loop through each list of YF Financials and clean CSV
    for file in list:
        
        # Read in CSV
        path = Path(file)
        csv = pd.read_csv(path, engine='python')
        
        # Clean CSV using step_1 function
        step = step_1(csv)
        
        # Add data from csv to df
        df = pd.concat([df,step], axis = 0)
        
        # Fill NaNs
        df = df.fillna(0)
        
        # Remove "\t" from column names
        df.columns = df.columns.str.strip()
        
        # Reset index
        df.reset_index(drop=True, inplace=True)
        
        # Drop useless column
        df.drop(columns='index',inplace=True)
    
    # Loop through columns and drop columns that have 0 for more than 20% of the data therein
    for column in df.columns[1:]:
        zeros = (df[column]==0).sum()
        if zeros >= 0.50*len(df.index):
            df.drop(columns = column, inplace = True)
        else:
            pass
    
    return df

In [10]:
# Create dataframe concatenated from respective categorical dataframes
def step_3(list):
    df = pd.DataFrame()
    tickers = []
    
    # Loop through each list of respective YF CSVs
    for list in list_of_lists:
        
        #Clean and concatenate CSV based on category
        frame = step_2(list)
        df = pd.concat([df,frame],axis=1)
    
    # Drop random NaN row?
    df = df.dropna()
    
    #Clean and update index for syntax
    for name in bs_list:
        tickers.append(name[:4])
    df.index = tickers
    
    for x in df.index:
        if x[3] == '_':
            df.rename(index={x:x[:3]},inplace=True)
        elif x[2] == '_':
            df.rename(index={x:x[:2]},inplace=True)
        elif x[1] == '_':
            df.rename(index={x:x[:1]},inplace=True)
        else:
            pass
    
    return df

In [11]:
#Running Functions and Compiling DFs
bz_df = step_3(list_of_lists)
bz_ipo_df = pd.concat([bz_df, ipo_df],axis=1)

In [12]:
bz_ipo_df.head(10)

,TotalAssets,CurrentAssets,CashCashEquivalentsAndShortTermInvestments,CashAndCashEquivalents,Receivables,PrepaidAssets,TotalNonCurrentAssets,OtherNonCurrentAssets,TotalLiabilitiesNetMinorityInterest,CurrentLiabilities,PayablesAndAccruedExpenses,Payables,AccountsPayable,CurrentAccruedExpenses,OtherCurrentLiabilities,TotalNonCurrentLiabilitiesNetMinorityInterest,TotalEquityGrossMinorityInterest,StockholdersEquity,CapitalStock,CommonStock,AdditionalPaidInCapital,RetainedEarnings,TotalCapitalization,CommonStockEquity,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,ShareIssued,OrdinarySharesNumber,NetPPE,GrossPPE,Properties,MachineryFurnitureEquipment,OtherProperties,Leases,AccumulatedDepreciation,GoodwillAndOtherIntangibleAssets,OtherIntangibleAssets,CurrentDebtAndCapitalLeaseObligation,CurrentDebt,CurrentDeferredLiabilities,CurrentDeferredRevenue,LongTermDebtAndCapitalLeaseObligation,LongTermDebt,LongTermCapitalLeaseObligation,OtherNonCurrentLiabilities,CapitalLeaseObligations,TotalDebt,AccountsReceivable,NonCurrentDeferredLiabilities,GainsLossesNotAffectingRetainedEarnings,OperatingCashFlow,CashFlowFromContinuingOperatingActivities,NetIncomeFromContinuingOperations,OperatingGainsLosses,StockBasedCompensation,ChangeInWorkingCapital,ChangeInReceivables,ChangeInPrepaidAssets,ChangeInPayablesAndAccruedExpense,ChangeInPayable,ChangeInAccountPayable,ChangeInAccruedExpense,FinancingCashFlow,CashFlowFromContinuingFinancingActivities,EndCashPosition,ChangesInCash,BeginningCashPosition,InterestPaidSupplementalData,IssuanceOfCapitalStock,FreeCashFlow,DepreciationAmortizationDepletion,DepreciationAndAmortization,ChangeInOtherWorkingCapital,InvestingCashFlow,CashFlowFromContinuingInvestingActivities,NetPPEPurchaseAndSale,PurchaseOfPPE,NetIssuancePaymentsOfDebt,NetLongTermDebtIssuance,LongTermDebtIssuance,LongTermDebtPayments,CapitalExpenditure,IssuanceOfDebt,RepaymentOfDebt,OtherNonCashItems,ChangesInAccountReceivables,NetOtherFinancingCharges,TotalRevenue,OperatingRevenue,OperatingExpense,SellingGeneralAndAdministration,GeneralAndAdministrativeExpense,OtherGandA,ResearchAndDevelopment,OperatingIncome,NetNonOperatingInterestIncomeExpense,OtherIncomeExpense,PretaxIncome,NetIncomeCommonStockholders,NetIncome,NetIncomeIncludingNoncontrollingInterests,NetIncomeContinuousOperations,DilutedNIAvailtoComStockholders,BasicEPS,DilutedEPS,BasicAverageShares,DilutedAverageShares,TotalOperatingIncomeAsReported,TotalExpenses,NetIncomeFromContinuingAndDiscontinuedOperation,NormalizedIncome,NetInterestIncome,EBIT,NetIncomeFromContinuingOperationNetMinorityInterest,TotalUnusualItemsExcludingGoodwill,TotalUnusualItems,NormalizedEBITDA,TaxRateForCalcs,TaxEffectOfUnusualItems,InterestExpenseNonOperating,OtherNonOperatingIncomeExpenses,InterestExpense,ReconciledDepreciation,TaxProvision,CostOfRevenue,GrossProfit,ReconciledCostOfRevenue,Industry,Offer Date,Shares (millions),Offer Price,Return,SPY 90D Return,10% Returns?
ABOS,"44,429,000","44,429,000","43,777,000","43,777,000","109,000","91,000",0,0,"63,020,000","6,367,000","864,000","531,000","531,000","333,000","5,503,000","56,653,000","-18,591,000","-18,591,000",0,0,"8,374,000","-26,965,000","-18,591,000","-18,591,000","-18,591,000","38,062,000","-18,591,000","-18,591,000","38,651,795","38,651,795",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"-7,450,000","-7,450,000","-7,325,000","-586,000","154,000","307,000","-79,000","53,000","189,000","308,000","308,000","-119,000","44,675,000","44,675,000","43,777,000","37,225,000","6,552,000",0,"44,675,000","-7,450,000",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"1,436,000","1,436,000","9,348,000","1,351,000","1,351,000","1,351,000","7,997,000","-7,912,000","1,000","586,000","-7,325,000","-7,911,000","-7,325,000","-7,325,000","-7,325,000","-7,911,000",-0.28,-0.28,"28,651,796","28,651,796","-7,912,000","9,348,000","-7,325,000","-7,911,000","1,000","-7,912,000","-7,325,000","586,000","586,000","-8,498,000",0,0,0,0,0,0,0,0,0,0,4,26,10.0,16.0,0.26,0.068748,1
ABSI,"8

In [13]:
bz_ipo_df.shape

(249, 136)

In [14]:
bz_ipo_df['Industry'] = bz_ipo_df['Industry'].astype(str)

In [15]:
bz_ipo_df['10% Returns?'] = bz_ipo_df['10% Returns?'].astype(str)

In [16]:
for column in bz_ipo_df.columns:
    bz_ipo_df[column] = bz_ipo_df[column].astype(str)
    bz_ipo_df[column] = bz_ipo_df[column].str.replace(',','').astype(float)

In [17]:
bz_ipo_df.dtypes

TotalAssets                                            float64
CurrentAssets                                          float64
CashCashEquivalentsAndShortTermInvestments             float64
CashAndCashEquivalents                                 float64
Receivables                                            float64
PrepaidAssets                                          float64
TotalNonCurrentAssets                                  float64
OtherNonCurrentAssets                                  float64
TotalLiabilitiesNetMinorityInterest                    float64
CurrentLiabilities                                     float64
PayablesAndAccruedExpenses                             float64
Payables                                               float64
AccountsPayable                                        float64
CurrentAccruedExpenses                                 float64
OtherCurrentLiabilities                                float64
TotalNonCurrentLiabilitiesNetMinorityInterest          

In [18]:
X = bz_ipo_df.iloc[:, 0:-1]
y = bz_ipo_df.iloc[:, -1]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [22]:
## train data
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(X_test))

In [23]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [24]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 135.
        self.layer_1 = nn.Linear(135, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [26]:
model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BinaryClassification(
  (layer_1): Linear(in_features=135, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [27]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [28]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.71084 | Acc: 54.333
Epoch 002: | Loss: 0.61764 | Acc: 67.000
Epoch 003: | Loss: 0.57693 | Acc: 72.000
Epoch 004: | Loss: 0.53527 | Acc: 77.667
Epoch 005: | Loss: 0.50171 | Acc: 78.333
Epoch 006: | Loss: 0.44679 | Acc: 84.667
Epoch 007: | Loss: 0.44266 | Acc: 86.333
Epoch 008: | Loss: 0.40481 | Acc: 87.333
Epoch 009: | Loss: 0.40070 | Acc: 89.667
Epoch 010: | Loss: 0.38183 | Acc: 89.333
Epoch 011: | Loss: 0.34523 | Acc: 91.667
Epoch 012: | Loss: 0.32050 | Acc: 93.333
Epoch 013: | Loss: 0.29966 | Acc: 91.333
Epoch 014: | Loss: 0.29166 | Acc: 94.667
Epoch 015: | Loss: 0.24882 | Acc: 94.333
Epoch 016: | Loss: 0.23909 | Acc: 93.667
Epoch 017: | Loss: 0.22285 | Acc: 94.000
Epoch 018: | Loss: 0.21373 | Acc: 95.000
Epoch 019: | Loss: 0.18645 | Acc: 95.333
Epoch 020: | Loss: 0.16027 | Acc: 97.667
Epoch 021: | Loss: 0.15035 | Acc: 99.333
Epoch 022: | Loss: 0.13281 | Acc: 97.667
Epoch 023: | Loss: 0.14292 | Acc: 98.333
Epoch 024: | Loss: 0.16407 | Acc: 97.333
Epoch 025: | Los

In [29]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [30]:
y_pred_list

[1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0]

In [31]:
confusion_matrix(y_test, y_pred_list)

array([[30,  4],
       [11, 38]], dtype=int64)

In [32]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.73      0.88      0.80        34
         1.0       0.90      0.78      0.84        49

    accuracy                           0.82        83
   macro avg       0.82      0.83      0.82        83
weighted avg       0.83      0.82      0.82        83

